# ESG Overview

Once a niche field, ESG has grown in prominence as a vital indicator for investors to assess the various industries and market sectors. Representing three primary standards for company valuation, ESG stands for Environmental, Social, and Governance criteria. The environmental metric focuses on the sustainable use of the Earth's resources, encompassing the impact on biodiversity to pollution and deforestation from business operations. On the other hand, the social metric concentrates on labor practices along with diversity, equity, and inclusion in the workforce among other social policies. Lastly, goverance is measured through how well a company abides by ethical and govermental policies, promoting transparent managerial practices. These three pillars of the ESG framework are collectively applied to capture the non-financial risks aspects of a company's operational scope. 

## Rationale of Comparing ESG Ratings

In order to assess the ESG performance of a company, ESG scores from independent third-party rating agencies are often factored in for socially motivated shareholders. However, as ESG investing gains more and more traction for screening investments, a major flaw in the ESG rating system must be addressed. Since there are no standardized regulations on how ESG indicators are to be evaluated by rating agencies, the scores are usually subjective to the weighting methodology employed by the specific agency. Therefore, ESG ratings tend to conflict with each other as a certain agency may weigh more heavily on sustainability for their assessment, while another may focus on goverance, which can lead to inconsistent results.

As a result, comparing ESG metrics and rating standards of different agenices paints a more clear picture that investors can use to screen prospective investments. With that goal in mind, I have created two web scraper tools below that will allow a user to exract a comapny's ESG metrics from both the Sustainalytics and S&P Global rating agencies. By analyzing ESG scoring features of the two rating standards, investors will be able to simultaneously access a comprehensive outlook of a company's ESG framework, empowering them to make conscientious investment decisions.



## Sustainalytics Web Scraper

The Sustainalytics Web Scraper provides the user with an aggregate ESG score ranging from 0 to 40+ with lower scores equating to lower ESG risk and vice versa. The scores are  further categorized all the way from negligible to severe risk. In addition, the web scraper also extracts the company ranking by industry and globally from Sustainalytics for further comparision.

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#acquires user input for specific company ESG rating url from Sustainalytics
url = input("Enter URL of Company ESG Rating from Sustainalytics: ")

#extracts and stores page content from Sustainalytics
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
results = requests.get(url, headers)
soup = BeautifulSoup(results.content, "html.parser")
soup = BeautifulSoup(soup.prettify(), "html.parser")

#scrapes relevant ESG information and rankings for the company
company = soup.find("div", class_ ="row company-name").get_text()
esg_score = soup.find("div", class_ ="col-6 risk-rating-score").get_text()
esg_assessment = soup.find("div", class_ ="col-6 risk-rating-assessment").get_text()
industry_group = soup.find("strong", class_ ="industry-group").get_text()
industry_position = soup.find("strong", class_ ="industry-group-position").get_text()
industry_total = soup.find("span", class_ ="industry-group-positions-total").get_text()
global_position = soup.find("strong", class_ ="universe-position").get_text()
global_total = soup.find("span", class_ ="universe-positions-total").get_text()
last_update = soup.find("div", class_ ="update-date").get_text()

#cleans data formatting and converts into proper data types
company = company.strip()
esg_score = float(esg_score.strip())
esg_assessment = esg_assessment.strip()
industry_group = industry_group.strip()
industry_position = int(industry_position.strip())
industry_total = int(industry_total.strip())
global_position = int(global_position.strip())
global_total = int(global_total.strip())
last_update = last_update.replace("Last Update:", "").strip()


#displays information output in a cohorent and logical format
print(f"\n\033[1mCompany: {company}\033[0m")
print(f"\nESG Rating: {esg_score}")
print(f"\nESG Risk Assessment: {esg_assessment}")
print(f"\n{industry_group} Industry Rank: {industry_position} out of {industry_total}")
print(f"\nGlobal Rank: {global_position} out of {global_total}")
print(f"\nLast Updated: {last_update}")


Enter URL of Company ESG Rating from Sustainalytics: https://www.sustainalytics.com/esg-rating/apple-inc/1007903183

Company: Apple, Inc.

ESG Rating: 16.7

ESG Risk Assessment: Low Risk

Technology Hardware Industry Rank: 223 out of 653

Global Rank: 2204 out of 15226

Last Updated: Jul 13, 2022


## S&P Global Web Scraper

With the S&P Global Web Scraper, the user can access a more finer breakdown of ESG metrics, albeit without rankings or a risk assessment. The aggregate ESG score for S&P Global ranges from 0 to 100, which is an average of the environmental, social, and governance metrics of the company. Finally, the web scraper displays a breakdown of ESG metrics and industry benchmarks in a tabular format.

In [3]:
#acquires user input for specific company ESG rating url from S&P Globa;
url = input("Enter URL of Company ESG Rating from S&P Global: ")

#extracts and stores page content from S&P Global
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
results = requests.get(url, headers)
soup = BeautifulSoup(results.content, "html.parser")
soup = BeautifulSoup(soup.prettify(), "html.parser")

#scrapes overview information and cleans the data formatting
company = soup.find("h2", class_ ="headline-3").get_text()
industry = soup.find("li", id ="company-industry").get_text()
esg_score = soup.find("p", id ="esg-score").get_text()
last_update = soup.find("p", id ="company-score-update").get_text()

company = company.strip()
industry = industry.strip()
esg_score = esg_score.strip()
last_update = last_update.strip().replace(". Updated annually or in response to major developments.", "")

#defines a function to specifically scrape rating metrics from the S&P Global page in a meaningful format
def extract_scores(score):
    score = str(score)
    
    company_location = score.rfind("data-score")
    company_score = (score[company_location + 11:company_location + 15]).strip().replace("\"", "")

    mean_location = score.rfind("data-avg")
    industry_mean = (score[mean_location + 9:mean_location + 13]).strip().replace("\"", "")

    max_location = score.rfind("data-max")
    industry_best = (score[max_location + 9:max_location + 13]).strip().replace("\"", "")

    rating_metrics = [int(company_score), int(industry_mean), int(industry_best)]
    return rating_metrics

#creates a dictionary to store breakdown of ESG ratings
esg_metrics ={"Rating Benchmark": [f"{company} Score", "Industry Mean", "Industry Best"],
             "Environmental": [],
             "Social": [],
             "Governance": []}

#extracts rating metrics from each ESG category
env = soup.find(id = "dimentions-score-env")
esg_metrics["Environmental"] += extract_scores(env)

soc = soup.find(id = "dimentions-score-social")
esg_metrics["Social"] += extract_scores(soc)

gov = soup.find(id = "dimentions-score-govecon")
esg_metrics["Governance"] += extract_scores(gov)

#displays output, inlcuding ESG rating breakdown in table format
print(f"\n\033[1mCompany: {company}\n")
print(f"\033[0m{industry}\n")
print(f"ESG Rating: {esg_score}")

df = pd.DataFrame(esg_metrics)
df = df.style.set_caption(f"{company} ESG Rating Breakdown")
display(df)

print(f"\n{last_update}\n")


Enter URL of Company ESG Rating from S&P Global: https://www.spglobal.com/esg/scores/results?cid=4004205

Company: Apple Inc.

Industry: THQ Computers & Peripherals and Office Electronics

ESG Rating: 37


,Rating Benchmark,Environmental,Social,Governance
0,Apple Inc. Score,61,24,29
1,Industry Mean,36,32,34
2,Industry Best,95,89,89



Last Updated: December 16, 2022

